In [1]:
from transformers import AutoTokenizer

In [2]:
tok=AutoTokenizer.from_pretrained("mrinaldi/Gettone")

In [16]:
document="Ciao  ˚     a tutti"

In [14]:
encoding = tok(document, return_offsets_mapping=True, add_special_tokens=False)

In [19]:
encoding['offset_mapping']

[(0, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 12),
 (12, 18)]

In [18]:
len(encoding['input_ids'])

10

In [82]:
from transformers import AutoTokenizer
from nltk.tokenize import PunktTokenizer

from typing import List, Dict, Union, Any
from nltk.tokenize import PunktTokenizer

class FastSentenceChunker:
    def __init__(self, language: str, chunk_size: int, tokenizer: Any):
        self.punkt = PunktTokenizer(language)
        self.tokenizer = getattr(tokenizer, "tokenizer", tokenizer) 
        self.chunk_size = chunk_size

    def _get_encoding_data(self, encoding) -> tuple:
        if hasattr(encoding, "ids"): 
            return encoding.ids, encoding.offsets
        return encoding["input_ids"], encoding["offset_mapping"]

    def _process(self, text: str, encoding) -> Dict[str, Any]:
        ids, offsets = self._get_encoding_data(encoding)
        sent_spans = self.punkt.span_tokenize(text)
        
        chunks, curr_chunk = [], []
        tok_idx, n_tokens = 0, len(ids)

        for _, sent_end in sent_spans:
            sent_toks = []
            while tok_idx < n_tokens and offsets[tok_idx][0] < sent_end:
                sent_toks.append(ids[tok_idx])
                tok_idx += 1
            
            # For very long sequences
            if len(sent_toks) > self.chunk_size:
                if curr_chunk: chunks.append(curr_chunk); curr_chunk = []
                for i in range(0, len(sent_toks), self.chunk_size):
                    sub = sent_toks[i : i + self.chunk_size]
                    if len(sub) == self.chunk_size: chunks.append(sub)
                    else: curr_chunk.extend(sub)
                continue

            # Normal length
            if len(curr_chunk) + len(sent_toks) > self.chunk_size:
                chunks.append(curr_chunk)
                curr_chunk = list(sent_toks)
            else:
                curr_chunk.extend(sent_toks)

        if curr_chunk: chunks.append(curr_chunk)

        ranges, cursor = [], 0
        flat_tokens = []
        for c in chunks:
            flat_tokens.extend(c)
            ranges.append((cursor, cursor + len(c)))
            cursor += len(c)
            
        return {"tokens": flat_tokens, "chunks": ranges}

    def __call__(self, document: str) -> Dict[str, Any]:
        encoding = self.tokenizer(document, return_offsets_mapping=True, add_special_tokens=False)
        return self._process(document, encoding)

    def batched(self, batch: List[str]) -> List[Dict[str, Any]]:
        encodings = self.tokenizer(
            batch, return_offsets_mapping=True, add_special_tokens=False, 
            verbose=False, padding=False, truncation=False
        )
        
        return [
            self._process(doc, {
                'input_ids': encodings['input_ids'][i],
                'offset_mapping': encodings['offset_mapping'][i]
            })
            for i, doc in enumerate(batch)
        ]

In [83]:
splitter=FastSentenceChunker('italian',64,tok)

In [55]:
document="""La Citroën AX è un'autovettura prodotta tra il 1986 ed il 1998 dalla casa automobilistica francese Citroën.
Storia
Genesi e debutto

Nel gennaio del 1981 il direttivo Citroën diede il via al progetto S9. Scopo di tale progetto fu quello di trovare un'erede ai modelli Visa ed LNA. La Visa, pur essendo in listino in quel momento da tre anni, montava soluzioni tecniche oramai obsolete, come il vecchio bicilindrico di origine 2CV. Quanto alla LNA, si trattava semplicemente di una riproposizione della 104 a 3 porte, dotata quindi della medesima carrozzeria, ma con lo stesso bicilindrico raffreddato ad aria. Quanto alla 2 CV, si scelse provvisoriamente di non sostituirla con il nuovo modello che sarebbe nato dal progetto S9, poiché rappresentava un'icona del marchio francese.

Per quanto riguarda il disegno della carrozzeria, furono vagliate molte proposte stilistiche provenienti da svariate personalità legate in un modo o nell'altro al Gruppo PSA, ma anche da alcuni atelier esterni tra cui quella di Art Blakeslee (all'epoca responsabile per il design della Talbot) e quella di Fergus Pollock, che propose una carrozzeria monovolume, all'epoca rivoluzionaria.
Prototipo ECO 2000 del 1982

La direzione della progettazione della carrozzeria della nuova utilitaria fu infine affidata a Trevor Fiore, incaricato di vagliare le varie proposte[1]. Venne inoltre avviata una ricerca di mercato presso la potenziale clientela della nuova vettura.

Durante i primi anni di progettazione, la casa francese presentò una piccola serie di concept, tutte denominate con la sigla ECO 2000, caratterizzate da una carrozzeria molto aerodinamica (Cx pari a 0.22), dimensioni compatte e buona abitabilità interna in rapporto agli ingombri esterni[2][3]. Da questi prototipi sarebbe derivata la futura utilitaria dove il fattore aerodinamico avrebbe permesso una riduzione dei consumi ed una maggior brillantezza nel comportamento su strada. La riduzione dei consumi era uno dei punti fermi ed imprescindibili del progetto, per questo venne data molta importanza anche al risparmio di peso.

Per quanto riguardava la meccanica, si scelse di rinunciare a motori tradizionali della Citroën per puntare sulle nuove unità quadricilindriche PSA; per la nuova utilitaria si scelse di realizzare una nuova gamma di motori direttamente derivata dalla vecchia serie X, realizzata dalla Peugeot ancora anni prima della sua fusione con la Citroën.

Il debutto avvenne al salone dell'automobile di Parigi del 1986, con il nome di AX, una sigla con la quale si intendeva dare continuità alla gamma Citroën del tempo, composta principalmente dall'ammiraglia CX e dalla media BX oltre che dalla 2CV e dalla Visa. In realtà, se la AX sostituì la LNA, non fece lo stesso per la Visa, poiché questa si trovava un gradino più in alto nella gamma e proponeva anche motorizzazioni di livello superiore e la configurazione a 5 porte, non presente al debutto della AX[2]. Pertanto, la Visa continuò ad essere prodotta ancora per un paio di anni prima di essere sostituita nel 1991 dalla ZX. """

In [84]:
splitter.batched([document,document,document,document])[0]['chunks']

[(0, 51),
 (51, 105),
 (105, 143),
 (143, 178),
 (178, 242),
 (242, 295),
 (295, 311),
 (311, 374),
 (374, 434),
 (434, 498),
 (498, 562),
 (562, 620),
 (620, 644)]

In [85]:
splitter(document)['chunks']

[(0, 51),
 (51, 105),
 (105, 143),
 (143, 178),
 (178, 242),
 (242, 295),
 (295, 311),
 (311, 374),
 (374, 434),
 (434, 498),
 (498, 562),
 (562, 620),
 (620, 644)]

In [69]:
batch=[document,document]
encodings = tok(
            batch, return_offsets_mapping=True, add_special_tokens=False, 
            verbose=False, padding=False, truncation=False
        )
encodings[0].ids

[5524,
 6427,
 5413,
 31025,
 5357,
 104,
 5350,
 6402,
 10030,
 15095,
 5480,
 16363,
 5409,
 5305,
 17885,
 5635,
 5305,
 12032,
 5641,
 6836,
 16118,
 6794,
 8731,
 6427,
 5413,
 31025,
 6927,
 6696,
 7818,
 5522,
 5273,
 5269,
 20521,
 5388,
 10133,
 7508,
 5275,
 19680,
 5305,
 5840,
 7938,
 6427,
 5413,
 31025,
 15541,
 5305,
 6233,
 5300,
 6955,
 5320,
 6993,
 10382,
 5387,
 5263,
 6451,
 6955,
 5712,
 5950,
 5263,
 8847,
 6402,
 5371,
 5372,
 5633,
 9139,
 6288,
 5332,
 5635,
 5361,
 94,
 7729,
 5524,
 6288,
 6635,
 7220,
 9134,
 5278,
 5508,
 21507,
 5278,
 5505,
 6730,
 5362,
 6194,
 8085,
 5780,
 10849,
 11088,
 10182,
 25412,
 9629,
 10284,
 5738,
 5462,
 5305,
 10856,
 22540,
 6700,
 116,
 6760,
 5263,
 9454,
 5488,
 83,
 15154,
 17636,
 5468,
 5361,
 94,
 7438,
 5323,
 18407,
 10902,
 5263,
 5376,
 8756,
 7076,
 5345,
 24361,
 5253,
 5568,
 6118,
 5738,
 21217,
 6217,
 5345,
 13476,
 16545,
 5565,
 14154,
 5389,
 5295,
 5450,
 6072,
 22540,
 6700,
 116,
 6760,
 26958,
 67

In [86]:
from transformers import AutoTokenizer

def test_splitter(document):
    tokenizer = AutoTokenizer.from_pretrained("mrinaldi/gettone")
    document=document.strip()
    # 2. Instantiate splitter
    chunker = FastSentenceChunker(
        language="italian",
        chunk_size=8,
        tokenizer=tokenizer
    )

    # 3. Reference input ids
    reference_ids = tokenizer(document, add_special_tokens=False)["input_ids"]

    # 4. Run splitter
    out = chunker(document)

    # 5. Reconstruct by slicing based on the emitted chunk ranges
    flat_tokens = out["tokens"]
    chunk_ranges = out["chunks"]

    reconstructed = []
    for start, end in chunk_ranges:
        # extract the exact part of the sequence belonging to each chunk
        reconstructed.extend(flat_tokens[start:end])
    # 6. Assert perfect reconstruction
    assert reconstructed == reference_ids


In [87]:
test_splitter(document)